In [2]:
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as seabornInstance 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import warnings
%matplotlib inline

In [1]:
!pwd

/home/adityadas96/waymo


In [16]:
train_dataset = pd.read_csv('/home/scg2151/waymo-project/csv_data/training/training_data_1_12.csv', names=["vx", "vy", "vz", "dx", "dy", "vfx", "vfy", "vfz", "afx", "afy", "afz", "num_v_labels", "ax", "ay", "az"])
test_dataset = pd.read_csv('/home/scg2151/waymo-project/csv_data/validation/validation_data_1_12.csv', names=["vx", "vy", "vz", "dx", "dy", "vfx", "vfy", "vfz", "afx", "afy", "afz", "num_v_labels", "ax", "ay", "az"])
train_dataset = train_dataset.drop(train_dataset.index[0])
test_dataset = test_dataset.drop(test_dataset.index[0])
print("Shape of the training data is", train_dataset.shape)
print("Shape of the validation data is", test_dataset.shape)

Shape of the training data is (17654, 15)
Shape of the validation data is (6947, 15)


In [17]:
X_train = train_dataset.iloc[:, :12]
X_test = test_dataset.iloc[:, :12]
print("Shape of the X_train is", X_train.shape)
print("Shape of the X_test is", X_test.shape)

Shape of the X_train is (17654, 12)
Shape of the X_test is (6947, 12)


In [18]:
Y_train = train_dataset.iloc[:,12:]
Y_test = test_dataset.iloc[:, 12:]
print("Shape of the Y_train is", Y_train.shape)
print("Shape of the Y_test is", Y_test.shape)

Shape of the Y_train is (17654, 3)
Shape of the Y_test is (6947, 3)


In [19]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, f1_score
import pickle

def grid_search(model_type, tuned_parameters, X_train, y_train, X_test, y_test):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        clf = GridSearchCV(model_type, tuned_parameters, cv=3)
        clf.fit(X_train, y_train)
        print("Best parameters set found on development set:")
        print("")
        print(clf.best_params_)
        print("")

        print("The model is trained on the full development set.")
        print("The scores are computed on the full evaluation set.")
        print("")
        y_true, y_pred = y_test, clf.predict(X_test)
        print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
        print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
        print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
        return y_pred, y_true

In [20]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()

tuned_parameters = [{
    'max_depth': [80, 90],
    'max_features': [2, 3],
    'n_estimators': [100, 200]
}]

y_pred, y_true = grid_search(model, tuned_parameters, X_train, Y_train, X_test, Y_test)

Best parameters set found on development set:

{'n_estimators': 200, 'max_features': 3, 'max_depth': 80}

The model is trained on the full development set.
The scores are computed on the full evaluation set.

Mean Absolute Error: 0.2245551169504908
Mean Squared Error: 0.15353712696332444
Root Mean Squared Error: 0.3918381387299154


In [15]:
import numpy

In [9]:
numpy.savetxt("val_pred.csv", y_pred, delimiter=",")

In [10]:
test_dataset.to_csv('val_truth.csv', index=False, header=False)